In [27]:
#Imports
import pandas as pd
import os

In [28]:
# Load the CSV data
file_path = 'Accounts_History_2021.csv'  # Update this path as needed
data = pd.read_csv(file_path)

In [29]:
# Step 1: Remove any leading/trailing whitespace from all string columns
data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [30]:
# Step 2: Remove rows where 'ticker' is NaN to clean extraneous text
data = data.dropna(subset=['Symbol'])  # 'Symbol' is renamed to 'ticker' later in Step 3

In [31]:
# Step 3: Rename columns
rename_columns = {
    'Run Date': 'transaction_date',
    'Account': 'portfolio_name',
    'Action': 'notes',
    'Symbol': 'ticker',
    'Description': 'asset_name',
    'Quantity': 'quantity',
    'Price': 'price',
    'Amount': 'transaction_amount',
    'Commission': 'commission',
    'Fees': 'fees'
}
data = data.rename(columns=rename_columns)


In [ ]:
# Step 4: Fill missing values in 'commission' and 'fees' columns with 0
data['commission'] = data['commission'].fillna(0)
data['fees'] = data['fees'].fillna(0)

In [ ]:
# Step 5: Delete unnecessary columns
columns_to_delete = [
    'Type', 'Exchange Quantity', 'Exchange Currency', 'Currency', 
    'Exchange Rate', 'Accrued Interest', 'Settlement Date'
]
data = data.drop(columns=columns_to_delete, errors='ignore')

In [ ]:
# Step 6: Reorder columns to match the specified order
final_columns_order = [
    'ticker', 'asset_name', 'quantity', 'price', 'transaction_amount', 
    'commission', 'fees', 'portfolio_name', 'transaction_date', 'notes'
]
data = data[final_columns_order]

In [ ]:
# Step 7: Convert 'transaction_date' to the correct format for SQL insertion (YYYY-MM-DD)
data['transaction_date'] = pd.to_datetime(data['transaction_date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')

In [ ]:
# Step 8: Save to a new CSV file with '_cleaned' suffix
cleaned_file_path = os.path.splitext(file_path)[0] + '_cleaned.csv'
data.to_csv(cleaned_file_path, index=False)

print(f"Cleaned file saved as: {cleaned_file_path}")

Cleaned file saved as: Accounts_History_2021v2.csv


In [46]:
# Step 9: Insert Data into PostgreSQL
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Step 9a: Load environment variables from .env file
load_dotenv('../server_credentials.env')

# Step 9b: Set up database connection using environment variables
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

# Create SQLAlchemy engine for database connection
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Step 9c: Insert data into the database
try:
    with engine.connect() as connection:
        data.to_sql('asset_ledger', con=connection, if_exists='append', index=False)
    # Step 9d: Confirm successful insertion
    print("Data successfully inserted into the database.")
except Exception as e:
    # Step 9d: Handle errors during insertion
    print(f"An error occurred during data insertion: {e}")


An error occurred during data insertion: (psycopg2.errors.ForeignKeyViolation) insert or update on table "asset_ledger" violates foreign key constraint "asset_ledger_ticker_fkey"
DETAIL:  Key (ticker)=(PLTR) is not present in table "russel_3000".

[SQL: INSERT INTO asset_ledger (ticker, asset_name, quantity, price, transaction_amount, commission, fees, portfolio_name, transaction_date, notes) VALUES (%(ticker__0)s, %(asset_name__0)s, %(quantity__0)s, %(price__0)s, %(transaction_amount__0)s, %(commis ... 4053 characters truncated ... , %(commission__21)s, %(fees__21)s, %(portfolio_name__21)s, %(transaction_date__21)s, %(notes__21)s)]
[parameters: {'price__0': 18.38, 'quantity__0': 21.0, 'transaction_amount__0': -385.93, 'commission__0': 0.0, 'transaction_date__0': '2021-12-14', 'ticker__0': 'PLTR', 'fees__0': 0.0, 'asset_name__0': 'PALANTIR TECHNOLOGIES INC CL A', 'portfolio_name__0': 'HSA 226909708', 'notes__0': 'YOU BOUGHT PALANTIR TECHNOLOGIES INC CL A (PLTR) (Cash)', 'price__1': 20.